## Prepare distance to infrastructure data

`cerf` provides two different options for calculating the distance to near suitable transmission infrastructure: 

1. Get the distance to the nearest substation and gas pipeline within the utility zone in which the power plant is being sited.

2. Get the distance to the nearest substation and gas pipeline regardless of whether or not they are within the utility zone in which the power plant is being sited.


## Explore some of the package data

##### View a data frame of the sample LMP data

In [ ]:
lmp_df = cerf.generate_random_lmp_dataframe()

lmp_df.head()


##### View the available suitability raster files contained in the package

In [ ]:
suitability_file_list = cerf.list_available_suitability_files()

suitability_file_list


##### View the sample configuration file for year 2010 as a dictionary

In [ ]:
config_file = cerf.load_sample_config(yr=2010)

config_file


##### Get the file name containing the sample utility zones raster

In [ ]:
zones_file = cerf.sample_utility_zones_raster_file()

zones_file
